In [1]:
!pip3 install requests openpyxl rdflib

You should consider upgrading via the 'D:\OneDrive - Concordia University - Canada\Computer Science\ENCS 691\project\Conupedia\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import json
import rdflib
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from IPython.core.display import display
from rdflib.namespace import RDF, RDFS, XSD, OWL
from rdflib import Namespace, URIRef, Graph, Literal



# Courses

In [3]:
session = HTTPBasicAuth('295', '50cb40a1af0752086248f8b05b1f88b5')

def get_information(url):
    response = requests.get(url, auth=session)
    response.encoding = response.apparent_encoding
    response = pd.read_json(response.text)
    response.set_index('ID', inplace=True)
    response = response[~response.duplicated(keep='first')]
    return response

# Course Catalog

We can obtain a list of courses in all catalogs by running the following code:

In [4]:
url = 'https://opendata.concordia.ca/API/v1/course/catalog/filter/*/*/*'
catalog = get_information(url)

In [5]:
display(catalog)
# catalog.to_excel('catalog.xlsx')

,title,subject,catalog,career,classUnit,prerequisites,crosslisted
ID,,,,,,,
26,Financial and Managerial Accounting,ACCO,220,UGRD,3.0,"Never Taken/Not Registered: ACCO213, ACCO21...",NaN
27,Introduction to Financial Accounting,ACCO,230,UGRD,3.0,"Never Taken/Not Registered: ACCO213, ACCO22...",NaN
28,Introduction to Managerial Accounting,ACCO,240,UGRD,3.0,"Never Taken/Not Registered: ACCO218, ACCO22...",NaN
35,Financial Reporting I,ACCO,310,UGRD,3.0,"Course Corequisite: One of (COMM305, ACCO218, ...",NaN
43,Financial Reporting II,ACCO,320,UGRD,3.0,"Course Prerequisite: One of (ACCO310, ACCO323)...",NaN
...,...,...,...,...,...,...,...
49226,"Feminism, Comedy and Social Change",WSDB,310,UGRD,3.0,,NaN
49227,"Feminisms, Tourism and Mobilities",WSDB,410,UGRD,3.0,Prerequisite: 30 credits including WSDB290 and...,NaN
50131,Feminist Perspectives on Human Rights,WSDB,300,UGRD,3.0,"Prerequisite: 15 credits including WSDB 290, 2...",NaN


## Course Descriptions

Course descriptions can be acquired by running the following code:

In [6]:
url = 'https://opendata.concordia.ca/API/v1/course/description/filter/*'
descriptions = get_information(url)
# display(descriptions)

Merge the descriptions and courses together

In [7]:
df = catalog.join(descriptions, on='ID')
# df.to_excel('hello.xlsx')

# RDF

We define the following prefixes:

In [8]:
SS = Namespace('http://www.securesea.ca/')
DBR = Namespace('http://dbpedia.org/resource/')
SCHEMA = Namespace('http://schema.org/version/6.0/#')

In [27]:
def to_rdf(series):
    g = Graph()
    g.bind('rdf', RDF)
    g.bind('schema', SCHEMA)
    g.bind('dbr', DBR)
    course = URIRef(f'http://www.securesea.ca/Courses/{str(series.name).zfill(5)}')
    g.add((course, RDF.type, SCHEMA.Course))
    g.add((course, SCHEMA.courseCode, Literal(series['subject'] + series['catalog'])))
    g.add((course, SCHEMA.name, Literal(series['title'])))
    g.add((course, SCHEMA.numberOfCredits, Literal(series['classUnit'], datatype=XSD.float)))
    g.add((course, SCHEMA.provider, DBR.Concordia_University))
    g.add((course, SCHEMA.coursePrerequisites, Literal(series['prerequisites'])))
    g.add((course, SCHEMA.description, Literal(series['description'])))

    return g

def to_graph(frame):
    dff = frame.apply(to_rdf, axis=1)
    g = Graph()
    g.bind('rdf', RDF)
    g.bind('schema', SCHEMA)
    g.bind('dbr', DBR)

    for _, value in dff.items():
        g += value
    g.serialize(destination="tbl.ttl")


to_graph(df)